In [1]:
import os
import time
import string
import nltk
import re
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer, PorterStemmer
from collections import defaultdict


In [2]:
# Ensure you have downloaded the NLTK stopwords dataset
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\pc\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
# Ensure the BOOKS directory exists at the same level as the script
books_dir = "./BOOKS"
if not os.path.exists(books_dir):
    os.makedirs(books_dir)
    


In [4]:

def preprocess_text(text):
    lemmatizer = WordNetLemmatizer()
    stemmer = PorterStemmer()
    stop_words = set(stopwords.words('english'))

    # Tokenize text
    tokens = word_tokenize(text)
    # Remove non-alphanumeric tokens and stopwords
    tokens = [token for token in tokens if token.isalnum() and token not in stop_words]
    # Lemmatize tokens
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    # Stem tokens
    tokens = [stemmer.stem(token) for token in tokens]
    
    # Join the tokens into a single string
    return " ".join(tokens)


In [5]:
# Function to read books and create Term-Document Incidence Matrix
def create_tdim(books_dir):
    # Get list of book files in the directory
    book_files = [f for f in os.listdir(books_dir) if f.endswith('.txt')]  # Assuming txt files
    book_texts = []
    
    for book_file in book_files:
        with open(os.path.join(books_dir, book_file), 'r', encoding='utf-8') as f:
            text = f.read()
            clean_text = preprocess_text(text)
            book_texts.append(clean_text)
    
    # Use CountVectorizer to create Term-Document Incidence Matrix
    vectorizer = CountVectorizer(binary=True)  # Binary for incidence matrix (0 or 1)

    tdim = vectorizer.fit_transform(book_texts)

    # Get feature names (terms)
    terms = vectorizer.get_feature_names_out()

    return tdim, terms, book_files

# Record the start time
start_time = time.time()

# Create Term-Document Incidence Matrix
tdim, terms, book_files = create_tdim(books_dir)

# Record the end time
end_time = time.time()

# Calculate elapsed time
elapsed_time = end_time - start_time

# Convert sparse matrix to dense format and create a DataFrame for better visualization
tdim_dense = tdim.toarray()
tdim_df = pd.DataFrame(tdim_dense, columns=terms, index=book_files)

# Output results
print(f"Term-Document Incidence Matrix shape: {tdim.shape}")
print(f"Number of terms: {len(terms)}")
print(f"Books processed: {book_files}")
print(f"Elapsed time: {elapsed_time:.2f} seconds")

# Display the Term-Document Incidence Matrix
print("\nTerm-Document Incidence Matrix:")
print(tdim_df)


Term-Document Incidence Matrix shape: (44, 20651)
Number of terms: 20651
Books processed: ['1. THE SONNETS.txt', '10. THE SECOND PART OF KING HENRY THE FOURTH.txt', '11. THE LIFE OF KING HENRY THE FIFTH.txt', '12. THE FIRST PART OF HENRY THE SIXTH.txt', '13. THE SECOND PART OF KING HENRY THE SIXTH.txt', '14. THE THIRD PART OF KING HENRY THE SIXTH.txt', '15. KING HENRY THE EIGHTH.txt', '16. THE LIFE AND DEATH OF KING JOHN.txt', '17. THE TRAGEDY OF JULIUS CAESAR.txt', '18. THE TRAGEDY OF KING LEAR.txt', '19. LOVES LABOURS LOST.txt', '2. ALLS WELL THAT ENDS WELL.txt', '20. THE TRAGEDY OF MACBETH.txt', '21. MEASURE FOR MEASURE.txt', '22. THE MERCHANT OF VENICE.txt', '23. THE MERRY WIVES OF WINDSOR.txt', '24. A MIDSUMMER NIGHTS DREAM.txt', '25. MUCH ADO ABOUT NOTHING.txt', '26. THE TRAGEDY OF OTHELLO THE MOOR OF VENICE.txt', '27. PERICLES PRINCE OF TYRE.txt', '28. KING RICHARD THE SECOND.txt', '29. KING RICHARD THE THIRD.txt', '3. THE TRAGEDY OF ANTONY AND CLEOPATRA.txt', '30. THE TRAGEDY O

In [6]:
# Function to build an inverted index
def build_inverted_index(tdim, terms, book_files):
    inverted_index = defaultdict(list)
    
    # Convert the sparse matrix to a dense format
    tdim_dense = tdim.toarray()

    for doc_index, doc_vector in enumerate(tdim_dense):
        for term_index, value in enumerate(doc_vector):
            if value == 1:  # If the term is present in the document
                inverted_index[terms[term_index]].append(book_files[doc_index])

    return inverted_index

# Record the start time
start_time = time.time()

# Build the inverted index
inverted_index = build_inverted_index(tdim, terms, book_files)

# Record the end time
end_time = time.time()

# Calculate elapsed time
elapsed_time = end_time - start_time

# Output results
print(f"Elapsed time for processing and building inverted index: {elapsed_time:.2f} seconds")

# Display a small portion of the inverted index
print("\nInverted Index (showing first 5 terms and their document occurrences):")
for term, documents in list(inverted_index.items())[:5]:
    print(f"Term: {term}, Documents: {documents}")

Elapsed time for processing and building inverted index: 0.18 seconds

Inverted Index (showing first 5 terms and their document occurrences):
Term: 10, Documents: ['1. THE SONNETS.txt', '15. KING HENRY THE EIGHTH.txt']
Term: 100, Documents: ['1. THE SONNETS.txt', '44. VENUS AND ADONIS.txt']
Term: 101, Documents: ['1. THE SONNETS.txt']
Term: 102, Documents: ['1. THE SONNETS.txt']
Term: 103, Documents: ['1. THE SONNETS.txt']


In [7]:
class Node:
    def __init__(self, label, docs=[]):
        self.label = label
        self.docs = list(set(docs))  # Ensure unique document indices
        self.left = self.right = None

    def merge(self, node):
        self.docs = list(set(self.docs + node.docs))  # Merge and ensure uniqueness
        return self

    def __str__(self):
        return f"{self.label} {self.docs}"

class Tree:
    def __init__(self):
        self.root = None

    def insert(self, node):
        if self.root is None:
            self.root = node
        else:
            self.__insert(self.root, node)

    def __insert(self, root, node):
        if root is None:
            root = node
        elif node.label == root.label:
            root = root.merge(node)
            root.docs = sorted(root.docs)
        elif node.label < root.label:
            root.left = self.__insert(root.left, node)
        else:
            root.right = self.__insert(root.right, node)
        return root
    
    def search(self, label):
        return self.__search(self.root, label)
    
    def __search(self, root, label):
        if root is None:
            return None
        if label == root.label:
            return root
        if label <= root.label:
            return self.__search(root.left, label)
        else:
            return self.__search(root.right, label)
        
    def get_documents_for_label(self, label):
        node = self.search(label)
        if node:
            return node.docs
        else:
            return None
        
    def display(self):
        self.__display(self.root)

    def __display(self, root):
        if root is None:
            return
        self.__display(root.left)
        print(root)
        self.__display(root.right)

In [8]:
def preprocess_text(text):
    lemmatizer = WordNetLemmatizer()
    stemmer = PorterStemmer()
    stop_words = set(stopwords.words('english'))

    text = text.lower()
    tokens = word_tokenize(text)
    tokens = [token for token in tokens if token.isalnum() and token not in stop_words]
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    tokens = [stemmer.stem(token) for token in tokens]
    
    return tokens

def build_bst(folder_path):
    # Initialize BST
    bst = Tree()

    # Iterate through each file in the folder
    for filename in os.listdir(folder_path):
        filepath = os.path.join(folder_path, filename)

        with open(filepath, 'r', encoding='utf-8') as file:
            text = file.read()

            # Preprocess the text
            processed_tokens = preprocess_text(text)

            # Extract book index from filename using regular expression
            match = re.search(r'\d+', filename)
            index = int(match.group()) if match else None

            # Create a node for each token and insert into BST
            if index is not None:
                for token in processed_tokens:
                    node = Node(token, [index])
                    bst.insert(node)

    return bst

In [9]:
if __name__ == "__main__":
    folder_path = "C:/Users/pc/Downloads/W1 - retrieval/W1 - retrieval/books"

    start_time = time.time()

    # Build the BST
    bst = build_bst(folder_path)

    end_time = time.time()

    # Display the BST
    bst.display()

    # Display elapsed time
    elapsed_time = end_time - start_time
    print(f"Elapsed Time: {elapsed_time} seconds")

1 [1, 13, 14, 15]
10 [1, 15]
100 [1, 44]
1000 [44]
1004 [44]
1009 [44]
101 [1]
1012 [44]
1016 [44]
102 [1]
1020 [44]
1024 [44]
1028 [44]
103 [1]
1033 [44]
1036 [44]
104 [1, 44]
1040 [44]
1044 [44]
1049 [44]
105 [1]
1053 [44]
1057 [44]
106 [1]
1060 [44]
1065 [44]
1069 [44]
107 [1]
1072 [44]
1078 [44]
108 [1, 44]
1081 [44]
1085 [44]
1088 [44]
109 [1]
1093 [44]
1096 [44]
11 [1]
110 [1]
1100 [44]
1105 [44]
1108 [44]
111 [1]
1112 [44]
1116 [44]
112 [1, 44]
1120 [44]
1124 [44]
1129 [44]
113 [1]
1132 [44]
1136 [44]
114 [1]
1141 [44]
1144 [44]
1148 [44]
115 [1]
1152 [44]
1156 [44]
116 [1, 44]
1160 [44]
1164 [44]
1168 [44]
117 [1]
1172 [44]
118 [1]
1180 [44]
1184 [44]
1189 [44]
119 [1]
1192 [44]
12 [1, 44]
120 [1, 44]
121 [1]
122 [1]
123 [1]
124 [1, 44]
125 [1]
126 [1]
127 [1, 44]
128 [1]
129 [1]
13 [1]
130 [1]
131 [1]
132 [1, 44]
133 [1]
134 [1]
135 [1]
136 [1, 44]
137 [1]
138 [1]
139 [1, 44]
14 [1]
140 [1]
141 [1]
142 [1]
143 [1]
144 [1, 44]
145 [1]
146 [1]
147 [1]
148 [1, 44]
149 [1]
15 [1]


In [10]:
class QueryParser:
    def __init__(self, tree):
        self.tree = tree
    
    def edit_distance_cumulative(self,s1, s2):
        # Get lengths of the input strings
        len1, len2 = len(s1), len(s2)

        # Initialize the matrix with dimensions (len1+1) x (len2+1)
        m = [[0] * (len2 + 1) for _ in range(len1 + 1)]

        # Fill the first column
        for i in range(1, len1 + 1):
            m[i][0] = i

        # Fill the first row
        for j in range(1, len2 + 1):
            m[0][j] = j

        # Fill the matrix based on the edit distance formula
        for i in range(1, len1 + 1):
            for j in range(1, len2 + 1):
                cost = 0 if s1[i - 1] == s2[j - 1] else 1
                m[i][j] = min(
                    m[i - 1][j - 1] + cost,  # Substitution
                    m[i - 1][j] + 1,        # Deletion
                    m[i][j - 1] + 1         # Insertion
                )

        # Return the edit distance, which is the bottom-right value in the matrix
        return m[len1][len2]

    def find_closest_word(self, word):
        all_labels = self.get_all_words_from_tree(self.tree.root)
        return min(all_labels, key=lambda w: self.edit_distance_cumulative(word, w))

    def get_all_words_from_tree(self, root):
        words = []
        if root:
            words += self.get_all_words_from_tree(root.left)
            words.append(root.label)
            words += self.get_all_words_from_tree(root.right)
        return words

    def search_word(self, word):
        # Search for the word in the tree
        node = self.tree.search(word)

        if node is None:
            # If the word is not found, find the closest word using edit distance
            closest_word = self.find_closest_word(word)
            node = self.tree.search(closest_word)

        return node.docs if (node and hasattr(node, 'docs')) else [] , node.label

    def negate_documents(self, operand):
        # If operand is a list of documents, negate them
        if isinstance(operand, list):
            all_documents = set(range(1, 45))  # Assuming 1 to 44 are the possible document indices
            return sorted(list(set(all_documents) - set(operand)))
        else:
            print("it must be a listof documents returned")

    def apply_operator(self, operators, operands):
        operator = operators.pop()

        if operator == 'AND':
            operand2 = operands.pop()
            operand1 = operands.pop()
            result = list(set(operand1).intersection(set(operand2)))
            operands.append(result)
        elif operator == 'OR':
            operand2 = operands.pop()
            operand1 = operands.pop()
            result = list(set(operand1).union(set(operand2)))
            operands.append(result)
        elif operator == 'NOT':
            operand = operands.pop()
            result = self.negate_documents(operand)
            operands.append(result)

    def parse_query(self, query):
        start_time = time.time()

        tokens = self.tokenize_query(query)
        result = sorted(self.parse_tokens(tokens))
        
        end_time = time.time()
        elapsed_time = end_time - start_time

        return result, elapsed_time

    def tokenize_query(self, query):
        # Tokenize the query
        query = re.sub(r'([()])', r' \1 ', query)
        tokens = query.split()

        return tokens

    def parse_tokens(self, tokens):
        # Process the query tokens
        operators = []
        operands = []
        i = 0

        while i < len(tokens):
            token = tokens[i]

            if token == '(':
                operators.append(token)
                i += 1
            elif token == 'AND' or token == 'OR' or token == 'NOT':
                operators.append(token)
                i += 1
            elif token == ')':
                # Handle closing parenthesis
                while operators and operators[-1] != '(':
                    self.apply_operator(operators, operands)
                operators.pop()  # Remove '('
                i += 1
            else:
                # Handle regular word
                operands.append(self.search_word(token)[0])
                i += 1

        while operators:
            self.apply_operator(operators, operands)

            
        return operands[0] if operands else []

In [16]:
# Example query
query = "canva"

# Parse and process the query
result, elapsed_time = query_parser.parse_query(query)

# Display the result and elapsed time
print("Query Result:", result)
print(f"Elapsed Time: {elapsed_time} seconds")

Query Result: [9, 10, 12]
Elapsed Time: 0.0 seconds
